In [21]:
import numpy as np
import pandas as pd

MET_MAP = {
    "sleep": 0.9,
    "cycling": 8.0,
    "sports": 7.0,
    "stationary": 1.3,         
    "light_activity": 3.0 
}

def label_to_met(label):
    return MET_MAP[label]

WINDOW_SECONDS = 10
WINDOW_HOURS   = WINDOW_SECONDS / 3600
WINDOW_MINUTES = WINDOW_SECONDS / 60


In [13]:
def load_labels(csv_path):
    df = pd.read_csv(csv_path)
    return df["true_label"].tolist(), df["pred_label"].tolist()

def convert_to_met(labels):
    return np.array([label_to_met(lbl) for lbl in labels])

def compute_met_errors(true_met, pred_met):
    mae = np.mean(np.abs(pred_met - true_met))
    rmse = np.sqrt(np.mean((pred_met - true_met)**2))
    return mae, rmse

def compute_met_hours(true_met, pred_met):
    true_h = np.sum(true_met * WINDOW_HOURS)
    pred_h = np.sum(pred_met * WINDOW_HOURS)
    diff_h = abs(true_h - pred_h)
    return true_h, pred_h, diff_h

def time_in_zones(met):
    zones = {
        "sedentary": np.sum(met < 1.5),
        "light":     np.sum((met >= 1.5) & (met < 3)),
        "moderate":  np.sum((met >= 3) & (met < 6)),
        "vigorous":  np.sum(met >= 6),
    }
    # convert windows to minutes
    return {z: zones[z] * WINDOW_MINUTES for z in zones}


In [ ]:
def run_met_evaluation(csv_path):
    true_labels, pred_labels = load_labels(csv_path)
    
    true_met = convert_to_met(true_labels)
    pred_met = convert_to_met(pred_labels)
    
    mae, rmse = compute_met_errors(true_met, pred_met)

    true_h, pred_h, diff_h = compute_met_hours(true_met, pred_met)

    true_z = time_in_zones(true_met)
    pred_z = time_in_zones(pred_met)

    print("========== MET Evaluation ==========")
    print(f"MAE (MET):   {mae:.2f}")
    print(f"RMSE (MET):  {rmse:.2f}\n")

    print(f"True MET-hours: {true_h:.2f}")
    print(f"Pred MET-hours: {pred_h:.2f}")
    print(f"Difference:     {diff_h:.2f}\n")

    true_z_rounded = {k: round(v, 2) for k, v in true_z.items()}
    pred_z_rounded = {k: round(v, 2) for k, v in pred_z.items()}

    print("Time-in-Zones (minutes):")
    print("  True:", true_z_rounded)
    print("  Pred:", pred_z_rounded)

    return {
        "mae": round(mae, 2),
        "rmse": round(rmse, 2),
        "true_met_hours": round(true_h, 2),
        "pred_met_hours": round(pred_h, 2),
        "diff_met_hours": round(diff_h, 2),
        "true_zones": true_z_rounded,
        "pred_zones": pred_z_rounded
    }


In [15]:
import pandas as pd

df = pd.read_csv("../Models/predictions_with_hmm.csv")
df.head()

,participant,timestamp,true_label,predicted_raw,predicted_smoothed
0,P101,2016-04-26 02:02:00,sleep,sleep,sleep
1,P101,2016-04-26 02:02:10,sleep,sleep,sleep
2,P101,2016-04-26 02:02:20,sleep,sleep,sleep
3,P101,2016-04-26 02:02:30,sleep,sleep,sleep
4,P101,2016-04-26 02:02:40,sleep,sleep,sleep


In [16]:
df = df.rename(columns={"predicted_smoothed": "pred_label"})

# Make sure true_label matches expected name (it already does)
df = df.rename(columns={"true_label": "true_label"})

# OPTIONAL: keep only the two columns for clarity
df_clean = df[["true_label", "pred_label"]]

# Save cleaned version for MET pipeline
df_clean.to_csv("../Models/hmm_cleaned_for_met.csv", index=False)

print("Saved cleaned file → hmm_cleaned_for_met.csv")
df_clean.head()

Saved cleaned file → hmm_cleaned_for_met.csv


,true_label,pred_label
0,sleep,sleep
1,sleep,sleep
2,sleep,sleep
3,sleep,sleep
4,sleep,sleep


In [25]:
run_met_evaluation("../Models/hmm_cleaned_for_met.csv")

========== MET Evaluation ==========
MAE (MET):   0.26
RMSE (MET):  0.78

True MET-hours: 1380.29
Pred MET-hours: 1297.71
Difference:     82.58

Time-in-Zones (minutes):
  True: {'sedentary': 38624.67, 'light': 0.0, 'moderate': 11207.83, 'vigorous': 845.83}
  Pred: {'sedentary': 40446.0, 'light': 0.0, 'moderate': 9799.67, 'vigorous': 432.67}


{'mae': 0.26,
 'rmse': 0.78,
 'true_met_hours': 1380.29,
 'pred_met_hours': 1297.71,
 'diff_met_hours': 82.58,
 'true_zones': {'sedentary': 38624.67,
  'light': 0.0,
  'moderate': 11207.83,
  'vigorous': 845.83},
 'pred_zones': {'sedentary': 40446.0,
  'light': 0.0,
  'moderate': 9799.67,
  'vigorous': 432.67}}